In [1]:
import numpy as np
import math

In [2]:
#electrode matrices
electrode_voltages = np.zeros((8,8))
electrode_positions = np.zeros((8,8,2))

#RGC matrices
on_parasol_responses = np.zeros((2*28,16)) #28 rows in hexagonal grid means 28 rows on a straight, but 56 rows staggered
on_parasol_positions = np.zeros((2*28,16,2)) #32 cols in hex grid means 32 staggered cols = 16 straight cols

In [3]:
#set positions

#electrodes:
#512 micron patch of retina and 70 micron electrode pitch
#odd rows start at x=0 and go to 0+70*7 = 490
#even rows start at x=512 and go down to 512-70*7 = 22 (x=22 to x=512 by 70)
#y values are evenly spread from y=16 to about y=496 (by 60)

y_positions = np.arange(16,496,60)

i = 0
while(i<np.shape(electrode_positions)[0]): #go through all rows, use np.arrange to evenly space electrodes
    #set y position for this row
    electrode_positions[i,:,1] = np.ones(8)*y_positions[i]
    
    #set x positions for each column in this row
    if(i%2 == 0):
        electrode_positions[i,:,0] = np.arange(22, 513, 70)
    else:
        electrode_positions[i,:,0] = np.arange(0, 491, 70)
    i+=1

    

#on parasol rgc's:
#512 micron patch of retina, 28x32 hexagonal grid (flat hexagon, tops/bottoms flat, sides pointy)
#28.5*height of hexagon = 512, h ~ 17.96 microns, y value increases by 1/2 h each row
#16*(1.5*width of hexagon)+3/4*width of hexagon = 512, w = 512/24.75 ~ 20.7 microns
#set step size = 1.5*w
#odd rows go x=w/2 to x=512-(w/2+3w/4) by 24, even rows go x=0+(w/2+3w/4) to x=512-w/2 by 24

h=512/28.5
w=512/24.75
y_positions = np.arange(h/2, 1+512-h/2, h/2)

i = 0
while(i<np.shape(on_parasol_positions)[0]): #go through all rows, use np.arrange to evenly space neurons
    #set y position for this row
    on_parasol_positions[i,:,1] = np.ones(16)*y_positions[i]
    
    #set x positions for each column in this row
    if(i%2 == 0):
        on_parasol_positions[i,:,0] = np.arange(w/2+3*w/4, 1+512-w/2, 3*w/2)
    else:
        on_parasol_positions[i,:,0] = np.arange(w/2, 1+512-(w/2+3*w/4), 3*w/2)
    i+=1


In [4]:
# get electrode voltages from image

# random data:

electrode_voltages = np.random.rand(8,8) * 8 #lets assume up to 8 mV for now
electrode_voltages

array([[1.81495984, 0.44319458, 6.10719383, 4.29604938, 5.60464112,
        3.97534789, 4.18350117, 6.5176567 ],
       [5.09366205, 0.3979583 , 1.86201172, 1.27799873, 0.82138522,
        7.25801704, 6.29599498, 2.58533063],
       [2.30855092, 0.13974179, 5.09721743, 3.47646544, 1.3195104 ,
        4.63840844, 5.41379362, 7.39117723],
       [2.17002654, 4.95901898, 3.23922139, 7.55177283, 4.80027561,
        4.74143728, 6.73248301, 3.53693438],
       [7.06881841, 3.74779223, 7.78923178, 4.57942288, 6.57576467,
        4.63331964, 4.99061498, 7.99683383],
       [4.02536086, 4.31284341, 6.36379764, 3.15208281, 4.781452  ,
        7.33101631, 7.13218497, 3.47170151],
       [4.33181104, 3.87139957, 2.17098068, 4.40568856, 5.8580314 ,
        4.93777457, 6.66583936, 5.3112449 ],
       [2.27851733, 6.51633896, 4.03761239, 3.20702211, 6.44150658,
        0.70389056, 1.1170427 , 4.02233891]])

In [5]:
# get response of ganglion cells


#spacial response

#loop through all electrodes, find "bounding box" (within ~30 microns of electrode in each direction),
#go through all neurons and add response to that electrode if in box otherwise go to next neuron to speed up simulation

i=0
j=0
while(i<np.shape(electrode_positions)[0]):
    while(j<np.shape(electrode_positions)[1]):
        x_elec = electrode_positions[i,j,0]
        y_elec = electrode_positions[i,j,1]
        x1 = x_elec - 35
        x2 = x_elec + 35
        y1 = y_elec - 35
        y2 = y_elec + 35
        
        voltage = electrode_voltages[i,j]
        
        #go through all neurons and simulate responses
        
        #on parasol simulation:
        a=0
        b=0
        while(a<np.shape(on_parasol_positions)[0]):
            while(b<np.shape(on_parasol_positions)[1]):
                x=on_parasol_positions[a,b,0]
                y=on_parasol_positions[a,b,1]
                if(x1<x and x<x2 and y1<y and y<y2):
                    #add to response since it's inside the range of the electrode
                    
                    #bottom left of golden page 4 sais response is based solely on Gaussian from electrode
                    #also included is std dev = 35 microns
                    #G(x,y) = 1/(2*pi*sigma^2) * e^(-(r^2)/(2*sigma^2))
                    
                    r = math.sqrt((x - x_elec)**2 + (y-y_elec)**2)
                    sigma = 35
                    G = 1/(2*math.pi*sigma**2) * math.e**(-(r**2)/(2*sigma**2))
                    #G = 1 * math.e**(-(r**2)/(2*sigma**2)) # G with amplitude 1
                    
                    on_parasol_responses[a,b] += G
                
                #otherwise just continue to next neuron
                
                b+=1
            b=0
            a+=1
        
        
        #other rgc simulation:
        a=0
        b=0
        #...
        
        j+=1
    j=0
    i+=1

In [ ]:
#difference of gaussians code

#Lecture 6: G(x,y) = 1/(2*pi*sigma^2) * e^(-(r^2)/(2*sigma^2))
#gaussian center std dev values are 10, 8, 4, and 3.5 microns (on par, off par, on mid, off mid)
#gaussian surround std dev values from golden are 1.15 * center std dev
#center peak has value 1, surround peak has value 0.375 for parasol and 0.5 for midget
#(all from the end of page 3 and beginning of page 4)

#gaussian with amplitude included: G(x,y) = a * e^(-(r^2)/(2*sigma^2))

#diff of gaussians with amplitude: DoG(x,y) = a_cent * e^(-(r^2)/(2*sig_cent^2))
# - a_sur * e^(-(r^2)/(2*sig_sur^2))

#r(x,y) = sqrt((x-x_elec)^2 + (y-y_elec)^2)

r = math.sqrt((x - x_elec)**2 + (y-y_elec)**2)
a_cent = 1
a_sur = 0.375
sig_cent = 10
sig_sur = sig_cent * 1.15
DoG = a_cent * math.e**(-(r**2)/(2*sig_cent**2)) - a_sur * math.e**(-(r**2)/(2*sig_sur**2))
on_parasol_responses[a,b] += DoG